## Load libs

In [66]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score

## Import data

In [74]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

In [75]:
insp.head(3)

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0
1,188889914,ZRV-00001406,2010,2010-10-01,22,4,0
2,188889907,ZRV-00001406,2010,2010-10-01,22,4,0


In [76]:
network.head(3)

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False


In [77]:
samplesub.head(3)

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0


## Create our full features

In [78]:
df = pd.get_dummies(network['Province'])

# join the encoded dataframe with original dataframe
network = network.join(df)

df1 = pd.get_dummies(network['GasType'])

# join the encoded dataframe with original dataframe
network = network.join(df1)

df2 = pd.get_dummies(network['BoolBridle'])

# join the encoded dataframe with original dataframe
network = network.join(df2)

df3 = pd.get_dummies(network['Material'])

# join the encoded dataframe with original dataframe
network = network.join(df3)

network = network.drop(columns=['GasType', 'BoolBridle', 'Material',"Province","Town"])

In [79]:
network.head(3)

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,Almeria,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,489616,2012,63.0,7.792,0.4,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,489645,2012,90.0,2.080,0.4,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,489646,2012,90.0,111.173,0.4,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [80]:
data = insp.merge(network, how='inner', on='PipeId')
data

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,YearBuilt,Diameter,Length,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
1,189311802,ZRV-00001972,2012,2012-10-01,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
2,189311802,ZRV-00001972,2014,2014-10-08,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
3,189311802,ZRV-00001972,2016,2016-10-14,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
4,189311802,ZRV-00001972,2018,2018-10-09,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,ZRV-00004581,2020,2020-12-31,24,4,0,1993,63.0,1.237,...,0,0,0,0,0,0,1,0,0,0
6345340,56907425,ZRV-00004581,2020,2020-12-31,24,4,0,2000,160.0,169.732,...,0,0,0,0,0,0,1,0,0,0
6345341,7843615,ZRV-00002112,2021,2020-12-31,24,4,0,1995,150.0,0.361,...,0,1,0,0,0,0,0,0,0,0
6345342,56922456,ZRV-00004581,2020,2020-12-31,24,4,0,1993,63.0,23.306,...,0,0,0,0,0,0,1,0,0,0


In [83]:
insp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6345344 entries, 0 to 6345343
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   PipeId          int64 
 1   MaintenanceId   object
 2   InspectionYear  int64 
 3   InspectionDate  object
 4   MonthsLastRev   int64 
 5   Severity        int64 
 6   Incidence       int64 
dtypes: int64(5), object(2)
memory usage: 338.9+ MB


In [88]:
data = data.drop(columns=["MaintenanceId","InspectionYear","InspectionDate","MonthsLastRev","Severity"], axis = 1)
data

,PipeId,Incidence,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,0,1993,63.0,1.237,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6345340,56907425,0,2000,160.0,169.732,4.000,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6345341,7843615,0,1995,150.0,0.361,0.025,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6345342,56922456,0,1993,63.0,23.306,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Create the data

In [81]:
finalsub = samplesub.merge(network, how='left', on='PipeId')
finalsub = finalsub.drop(columns="Incidence")
finalsub

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,Almeria,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,446859944,2012,110.0,44.286,0.150,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,428124500,2012,160.0,110.715,0.150,2,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,438428871,2012,110.0,13.575,0.150,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,429034569,2012,110.0,327.689,0.150,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,411184477,2007,200.0,932.762,0.150,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,2004,110.0,7.656,0.025,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909729,235426673,2004,110.0,4.383,0.025,2,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909730,235426707,2004,110.0,2.292,0.025,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909731,190956601,2004,90.0,87.809,4.000,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Secret step

In [89]:
df = data
df = df.sample(frac=1)
incidence_df = df.loc[df['Incidence'] == 1]
non_incidence_df = df.loc[df['Incidence'] == 0][:12328]
normal_distributed_df = pd.concat([incidence_df, non_incidence_df])
new_df = normal_distributed_df.sample(frac=1, random_state=42)
new_df

,PipeId,Incidence,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
1566136,133290209,0,1993,90.0,11.339,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
92958,287837696,0,1901,200.0,3.923,0.025,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6133312,30085087,1,2016,63.0,50.300,1.700,2,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5491049,37374530,0,1950,90.0,0.450,0.025,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2131367,190934597,0,2004,101.6,2.504,16.000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5445072,332113275,0,2008,90.0,64.407,0.150,4,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6072750,45977323,1,2005,90.0,25.758,0.150,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3674798,191459442,1,1992,63.0,10.724,4.000,2,0,0,0,...,0,0,0,0,0,0,1,0,0,0
277213,189671098,0,2000,90.0,38.460,0.150,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


# Hypertune the model

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(new_df.loc[:, new_df.columns != "Incidence"], new_df["Incidence"], test_size=0.10, random_state=None)


# #dtrain = xgb.DMatrix(X_train, label=y_train)
# #dtest = xgb.DMatrix(X_test, label=y_test)


# xgb_clf = xgb.XGBClassifier(max_depth = 3, learning_rate = 0.01, n_estimators = 100)
# xgb_clf.fit(X_train, y_train)


# y_pred = xgb_clf.predict_proba(X_test)[:, 1]
# auc = roc_auc_score(y_test, y_pred)
# print("Test set AUC: ", auc)

# results_1 = xgb_clf.predict(finalsub)


In [91]:
X_train, X_test, y_train, y_test = train_test_split(new_df.loc[:, new_df.columns != "Incidence"], new_df["Incidence"], test_size=0.25, random_state=42)

# Convert the training and test sets into DMatrix objects for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define the grid of hyperparameters to search
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300]
}

# Perform grid search with cross-validation
xgb_clf = xgb.XGBClassifier()
grid_search = GridSearchCV(xgb_clf, params, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# Print the best parameters and the best AUC score
print("Best parameters: ", grid_search.best_params_)
print("Best AUC score: ", grid_search.best_score_)

# Evaluate the model on the test set
y_pred = grid_search.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
print("Test set AUC: ", auc)

results_1 = grid_search.predict(finalsub)

Best parameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best AUC score:  0.9101265051047689
Test set AUC:  0.9081903748890919


## Predict the kaggle results

In [92]:
results_1 = grid_search.predict(finalsub)
results1df = pd.DataFrame(results_1)
results1df = pd.DataFrame(results_1)

samplesub_sub = samplesub.drop(columns = "Incidence")

finalfinal = samplesub_sub.merge(results1df, left_index = True,right_index=True)
finalfinal.rename(columns={0: "Incidence"}, inplace=True)
finalfinal

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0
3,429034569,0
4,411184477,0
...,...,...
909728,235426649,0
909729,235426673,0
909730,235426707,0
909731,190956601,0


## Create the submission file

In [93]:
finalfinal.to_csv("newcatboost.csv", index=False)